<a href="https://colab.research.google.com/github/eatlc/FPGA-4x2_mux/blob/main/TidyDataMLv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [360]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [361]:
#kullandığım csv: tidy_data_test.csv
from google.colab import files
uploaded = files.upload()

Saving tidy_data_test.csv to tidy_data_test (1).csv


In [362]:
#test için kullanacağı verileri sildim bunlar tidy_data_v2 deki 85,94,112 TD verileri ve ASD olarak 
#burada kullanmadığım başlangı verilerinden 13 20 25i aldım
df=pd.read_csv('tidy_data_test.csv')  
df.shape

(160, 63)

In [363]:
#data daki idx değerlerini işime yaramadığı için sildim.
del df['idx']

#aşağıdaki datalarda TD çocuklar için boş kalan kısımlar var bunları
#değişkenleri cinsinden yani int veya float olmak üzere 0 lar ile doldurdum
df["FixGeo"].fillna(0, inplace = True)
df['ET1_Age'].fillna(0, inplace=True)
df['ETage'].fillna(0, inplace = True)
df['Fixation_Duration_Geometric_AOI'].fillna(0, inplace = True)
df['Fixation_Duration_Social_AOI'].fillna(0, inplace = True)
df['Total_Fixation_Time'].fillna(0, inplace = True)
df['Percent_Fixation_Social'].fillna(0, inplace = True)
df['Percent_Fixation_Geometric'].fillna(0, inplace = True)
df['ADOS_CoSoTot'].fillna(int(0), inplace = True)
df['ADOSatscan_age'].fillna(0, inplace = True)
df['ADOSatscan_module'].fillna("0", inplace = True)

#ASD ve TD değerlerinin dengeli olması için başlangıçtan 50 tane ASDli
#datayı kullanmadım
df=df.iloc[50:164]

#CaseControl değerlerini int e çevirdim 
#ASD:1 TD:0 olarak kaydettim
df['CaseControl'].replace({'ASD':int(1), 'TD':int(0)},inplace=True)

#ADOSatscan_module sütununda Module1, Module2, Toddler ve boş satırlar vardı
#bunları kullanabilmek aşağıdaki fonksiyon ile hepsini sütun haline getirdim
#ve ADOSatscan sütununu artık bu şekilde kullanacağım için sildim.
#concat satırları sütun yapıyor
ADOSatscan = pd.get_dummies(df['ADOSatscan_module'], drop_first = True)
df = pd.concat([df, ADOSatscan], axis=1)
del df['ADOSatscan_module']

#cinsiyet değişkenini de kadın:1 erkek:0 yaptım
df['sex'].replace({'F':int(1),'M':int(0)},inplace=True)


In [364]:
#burada ASD ve TD verlerimim dengesine baktım, birbirlerine yakın olması tavsiye edilir.
df['CaseControl'].value_counts(normalize = True)

1    0.536364
0    0.463636
Name: CaseControl, dtype: float64

In [365]:
from sklearn.model_selection import train_test_split  
# %60 train, %20 validation, %20 test
seed=111   
X = df.drop('CaseControl', axis=1)
y = df['CaseControl']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, 
                                                    random_state = seed)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, 
                                                    random_state = seed)



In [366]:
X.shape

(110, 63)

In [367]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 50 to 159
Data columns (total 63 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   sex                              110 non-null    int64  
 1   scan_age                         110 non-null    float64
 2   FixGeo                           110 non-null    float64
 3   ETage                            110 non-null    float64
 4   ET1_Age                          110 non-null    float64
 5   Fixation_Duration_Geometric_AOI  110 non-null    float64
 6   Fixation_Duration_Social_AOI     110 non-null    float64
 7   Total_Fixation_Time              110 non-null    float64
 8   Percent_Fixation_Geometric       110 non-null    float64
 9   Percent_Fixation_Social          110 non-null    float64
 10  meanFD                           110 non-null    float64
 11  meanDVARSwavelet                 110 non-null    float64
 12  meanDVARSraw         

In [368]:
#Lojistik regresyon

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(random_state = seed)

lr_hyp = {'C':[0.001, 0.01, 0.1, 1, 10, 100]}

lr_cv= GridSearchCV(lr, lr_hyp, cv = 5)

lr_cv.fit(X_train, y_train)

print(lr_cv.best_score_)
print(lr_cv.best_estimator_)


1.0
LogisticRegression(C=0.01, random_state=111)


In [369]:
#Random Forrest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
RF = RandomForestClassifier(random_state=seed)
RF_hyp = {'n_estimators':[5, 10, 20, 50, 100, 200],
          'max_depth':[None, 2, 5, 10, 15, 20]}

RF_cv = GridSearchCV(RF, RF_hyp, cv = 5)
RF_cv.fit(X_train, y_train)

print(RF_cv.best_score_)
print(RF_cv.best_estimator_)

1.0
RandomForestClassifier(n_estimators=50, random_state=111)


In [370]:
# SVM

from sklearn.svm import SVC

sv = SVC(random_state = seed)
sv_hyp = {'C': [0.1, 1, 10], 'kernel':['linear', 'rbf']}

svm_cv = GridSearchCV(sv, sv_hyp, cv=5)
svm_cv.fit(X_train, y_train)
           
print(svm_cv.best_score_)
print(svm_cv.best_estimator_)

1.0
SVC(C=0.1, kernel='linear', random_state=111)


In [371]:
#Bagging Sınıflandırma

from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier(random_state=seed)

bc_hyp = {'n_estimators': [5, 10, 20, 50, 100, 200]}

bc_cv = GridSearchCV(bc, bc_hyp, cv=5)

bc_cv.fit(X_train, y_train)

print(bc_cv.best_score_)
print(bc_cv.best_estimator_)

1.0
BaggingClassifier(n_estimators=5, random_state=111)


In [372]:
#Performansların değerlendirilmesi ve modelin seçimi

def model_evaluation(model, features, labels):
  pred = model.predict(features)
  score = accuracy_score(y_val, pred)
  print('Score globale du modele:' , round(score, 3))

In [373]:
models = [lr_cv.best_estimator_, RF_cv.best_estimator_,
          svm_cv.best_estimator_, bc_cv.best_estimator_]

for model in models:
  print('Model' + str(model))
  model_evaluation(model, X_val, y_val)
  print('-'*90)

ModelLogisticRegression(C=0.01, random_state=111)
Score globale du modele: 1.0
------------------------------------------------------------------------------------------
ModelRandomForestClassifier(n_estimators=50, random_state=111)
Score globale du modele: 1.0
------------------------------------------------------------------------------------------
ModelSVC(C=0.1, kernel='linear', random_state=111)
Score globale du modele: 1.0
------------------------------------------------------------------------------------------
ModelBaggingClassifier(n_estimators=5, random_state=111)
Score globale du modele: 1.0
------------------------------------------------------------------------------------------


In [374]:
#En iyi modelin test verileriyle performansı 

accuracy_score(y_test, bc_cv.best_estimator_.predict(X_test))

1.0

In [375]:
#test iin kullanacağım değerli, silmediğim csv den alıyorum 
#kullandığım csv: tidy_data_v2.csv
uploaded2 = files.upload()

Saving tidy_data_v2.csv to tidy_data_v2 (8).csv


In [376]:
dfi=pd.read_csv('tidy_data_v2.csv')

#yukarıdaki gibi önce cvs yi kullanabileceğim hale getiriyorum
#boş verileri 0la dolduruyorum, bir sütunu satır yapıyorum vs. yukarıdaki gibi

del dfi['idx']
dfi["FixGeo"].fillna(0, inplace = True)
dfi['ET1_Age'].fillna(0, inplace=True)
dfi['ETage'].fillna(0, inplace = True)
dfi['Fixation_Duration_Geometric_AOI'].fillna(0, inplace = True)
dfi['Fixation_Duration_Social_AOI'].fillna(0, inplace = True)
dfi['Total_Fixation_Time'].fillna(0, inplace = True)
dfi['Percent_Fixation_Social'].fillna(0, inplace = True)
dfi['Percent_Fixation_Geometric'].fillna(0, inplace = True)
dfi['ADOS_CoSoTot'].fillna(int(0), inplace = True)
dfi['ADOSatscan_age'].fillna(0, inplace = True)
dfi['ADOSatscan_module'].fillna("0", inplace = True)

ADOSatscani = pd.get_dummies(dfi['ADOSatscan_module'], drop_first = True)
dfi = pd.concat([dfi, ADOSatscani], axis=1)
del dfi['ADOSatscan_module']
dfi['sex'].replace({'F':int(1),'M':int(0)},inplace=True)

In [377]:
#Buradan test için kullanacağım, train veri setinde silinmiş olan 
#satırları çekip birleştiriyorum. 
#3 tane ASD, 3 tane TD aldım

df1=dfi.iloc[13:14,:]
df2=dfi.iloc[20:21,:]
df3=dfi.iloc[25:26,:]
df4=dfi.iloc[85:86,:]
df5=dfi.iloc[94:95,:]
df6=dfi.iloc[112:113,:]

df_son=pd.concat([df1,df2,df3,df4,df5,df6])
df_son

,CaseControl,sex,scan_age,FixGeo,ETage,ET1_Age,Fixation_Duration_Geometric_AOI,Fixation_Duration_Social_AOI,Total_Fixation_Time,Percent_Fixation_Geometric,...,IC10_IC28,IC11_IC21,IC11_IC26,IC11_IC28,IC21_IC26,IC21_IC28,IC26_IC28,Module1,Module2,Toddler
13,ASD,0,30.423057,87.40,39.0,39.0,41.82,6.03,47.85,87.40,...,0.272466,-0.520472,-0.127712,-0.150710,-0.012135,-0.159009,0.029039,1,0,0
20,ASD,0,34.299337,42.91,25.0,25.0,13.79,18.34,32.13,42.91,...,0.248428,-0.479888,-0.184320,0.106633,-0.110601,-0.284718,0.251845,1,0,0
25,ASD,1,30.028856,62.94,41.0,41.0,37.96,22.35,60.31,62.94,...,0.530700,-0.028112,-0.098700,-0.394631,-0.073807,-0.124685,0.112588,0,0,1
85,TD,0,13.273101,26.14,12.0,12.0,11.34,32.04,43.38,26.14,...,-0.158176,-0.241302,-0.028340,0.098889,-0.102713,0.089986,-0.079520,0,0,0
94,TD,1,41.297710,35.50,40.0,40.0,18.08,32.84,50.92,35.50,...,0.277143,-0.270500,-0.267857,0.050070,-0.197890,-0.098510,-0.128530,0,0,0
112,TD,0,44.089873,4.39,30.0,30.0,2.63,57.26,59.89,4.39,...,-0.273863,-0.379613,-0.012803,0.096570,-0.038959,-0.213739,0.182977,0,0,0


In [378]:
#test edbilmek için CaseControl kısmını siliyorum.
#yukarıda hangisi ASD hangisi TD bilmek için gösterdim
del df_son['CaseControl']

In [383]:
#istediğim satırı alıyorum
test=df_son.iloc[0:1,:]
test

,sex,scan_age,FixGeo,ETage,ET1_Age,Fixation_Duration_Geometric_AOI,Fixation_Duration_Social_AOI,Total_Fixation_Time,Percent_Fixation_Geometric,Percent_Fixation_Social,...,IC10_IC28,IC11_IC21,IC11_IC26,IC11_IC28,IC21_IC26,IC21_IC28,IC26_IC28,Module1,Module2,Toddler
13,0,30.423057,87.4,39.0,39.0,41.82,6.03,47.85,87.4,12.6,...,0.272466,-0.520472,-0.127712,-0.15071,-0.012135,-0.159009,0.029039,1,0,0


In [384]:
#aldığım veriyi predicte gönderiyorum ve tahmin sonucumu alıyorum
pred=svm_cv.predict(test)
print(pred)

[1]
